# Importing Libraries

In this project, we are going to use common libraries below


In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import requests 
import xlsxwriter
import math

# Importing S&P 500 Symbol File
We first import a static file containing 505 different symbols for each company in S&P 500.

In [13]:
stocks=pd.read_csv('sp_500_stocks.csv')

# Acquire API Token
In this project for illustration purposes, we are going to use a sandbox API token that is free from IEX Cloud. 

In [14]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

# Split Symbols Into Subgroups

Because of time effienciency, it's better to use a batch API call rather than a single invidual API call for each symbol. 

We first need to break down the 505 symbols into 5 groups with 100 symbols per group. 

In [15]:
def chunks (lst,n): # split a list into sublist
    for i in range(0,len(lst),n):
        yield lst[i:i+n]
symbol_groups=list(chunks(stocks['Ticker'],100))
symbol_strings=[]
for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

# Creating A Column List for DataFrame
The following is to create a list of each coloumns for the data frame we are going to work on. 

In [18]:
rv_columns=['Ticker',
            'Price',
            'Numbers of Shares to Buy',
            'Price-to-Earnings Ratio',
            'PE Percentile',
            'Price-to-Book Ratio',
            'PB Percentile',
            'Price-to-Sale Ratio',
            'PS Percentile',
            'EV/EBITDA',
            'EV/EBITDA Percentile',
            'EV/GP',
            'EV/GP Percentile',
            'RV'] #Robust Value

# Requesting Batch API Calls

Now we have 5 groups of symbols. The next step is to make a batch API calls for each group of symbols to get different ratios which consist of 

* Price-Earning Ratio
* Price-to-book Ratio
* Price-to-sales Ratio
* EV(Enterprise Value)-to-EBITDA(Earning Before Interest,Taxes,Depreciation and Ammortization)
* EV-to-Gross Profit

We later then append these ratios into our dataframe.

In [21]:
rv_dataframe=pd.DataFrame(columns=rv_columns)
for x in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={x}&types=advanced-stats,quote&token={IEX_CLOUD_API_TOKEN}'
    data=requests.get(batch_api_call_url).json()
    for y in x.split(","):
        try:
            EV_EBITDA=data[y]['advanced-stats']['enterpriseValue']/data[y]['advanced-stats']['EBITDA']
        except TypeError:
            EV_EBITDA=np.nan
        try:
            EV_GP=data[y]['advanced-stats']['enterpriseValue']/data[y]['advanced-stats']['grossProfit']
        except TypeError:
            EV_GP=np.nan
        rv_dataframe=rv_dataframe.append(pd.Series(
            [y,
             data[y]['quote']['latestPrice'],
             "N/A",
             data[y]['quote']['peRatio'],
            "N/A",
             data[y]['advanced-stats']['priceToBook'],
             "N/A",
             data[y]['advanced-stats']['priceToSales'],
             'N/A',
             EV_EBITDA,
             'N/A',
             EV_GP,
             'N/A',
             'N/A'
            ],index=rv_columns
        ),ignore_index=True)

rv_dataframe.dropna(inplace=True)

# Calculating Each Ratios' Percentile and RV Value
We use stats.percentileofscore function to compute percentiles for each ratio. Then we use apply method to calculate the mean of all percentiles for each symbol to the RV(Robust Value).

In [23]:
rvs=[
    'PE',
    'PB',
    'PS',
    'EV/EBITDA',
    'EV/GP'
]

rv_columns1=['Ticker',
            'Price',
            'Numbers of Shares to Buy',
            'PE Ratio',
            'PE Percentile',
            'PB Ratio',
            'PB Percentile',
            'PS Ratio',
            'PS Percentile',
            'EV/EBITDA Ratio',
            'EV/EBITDA Percentile',
            'EV/GP Ratio',
            'EV/GP Percentile',
            'RV']
rv_dataframe.columns=rv_columns1
for x in rv_dataframe.index:
    for y in rvs:
        rv_dataframe.loc[x,f'{y} Percentile']=stats.percentileofscore(rv_dataframe[f'{y} Ratio'].astype(np.float64),rv_dataframe.loc[x,f'{y} Ratio'])

rv_dataframe['RV']=rv_dataframe[[f'{y} Percentile' for y in rvs]].apply(np.mean,axis=1)
rv_dataframe.columns=rv_columns

# Sorting by RV and Select The Top 50 Stocks

In [26]:
rv_dataframe.sort_values('RV',ascending=True,inplace=True)
rv_dataframe.reset_index(inplace=True,drop=True)
rv_dataframe=rv_dataframe[:50]

# Caulating Number of Shares to Buy

We define how many shares to buy for each stock based on the portfolio value you enter. And we enter those value into "Numbers of Shares to Buy" variable from rv_dataframe using apply method.

In [28]:
def portfolio_input():
    global portfolio_size
    portfolio_size=input('Please Enter Your Portfolio Value:')
    try:
        float(portfolio_size)
    except:
        portfolio_size=input("Please enter a float or an integer: ")
portfolio_input()
position_size=float(portfolio_size)/len(rv_dataframe)
rv_dataframe['Numbers of Shares to Buy']=(position_size/rv_dataframe['Price']).apply(math.floor)


Please Enter Your Portfolio Value:1000000


<ipython-input-28-816f8951a6fa>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv_dataframe['Numbers of Shares to Buy']=(position_size/rv_dataframe['Price']).apply(math.floor)


# Saving into Excel

In [31]:
writer=pd.ExcelWriter('RobustValue.xlsx',engine='xlsxwriter')
rv_dataframe.to_excel(writer,'Value',index=False)
writer.save()